In [1]:
import numpy as np
import numpy,scipy,corner
import nestle
import astropy,warnings
from scipy.interpolate import interp1d as interp
from astropy.table import Table
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal as multi_gauss
#---------------------------------------------------------------------
import Library_GraphTwoDimensionDensityColorMap
#---------------------------------------------------------------------
warnings.simplefilter('ignore')
print ("done importing")

done importing


**Generate a set of true Datapoints**
==


In [2]:
#Set the variables for the 'real' model
v = 50
g = 9.8
def my_model( v, g, t):
    return v*t -.5*g*t**2

In [9]:
#Pick some times of which to observe the height
times=np.linspace(.1,10,20) # measurement times

#figure out the heights which are "real" for the given times above
perfect_data=my_model(v,g,times)


datapoint_count = len(times)

**Modify the data with height error only**
==

In [12]:
#Generate 1D dataset with measurement error
#   By 1D, we mean that we have only error in the height measurement. 
#   We have NO time measurement error   



#Introduce error into the "perfect" data using the experiment error
measurement_uncertainty = 10
errors=[measurement_uncertainty]*datapoint_count

heights=np.random.normal(perfect_data,errors)
my_data_1D=Table([times,heights,errors],
              names=('time','height','error'))

#print( my_data_1D )

#Write the generated data to a file on the disk of the computer
astropy.io.ascii.write( my_data_1D, "1D_Generated_Data.astropydat" , overwrite = True )

In [14]:
#(SECRET)
#Generate 1D uncertainty dataset with a model that will not work with
#   a simple chi-square minimization.
#   By 1D, we mean that we have only error in the height measurement. 
#   We have NO time measurement error   .

#Set the variables for our model, and the model itself
bad_v = 50
bad_g = -10
def my_bad_model( v,g, t):
    return v*t -.5*g**2*t**2

#Generate "perfect" data with the bad model.
perfect_bad_data=my_bad_model(bad_v,bad_g,times)
#Introduce error into the "perfect" data using the experiment error
heights_bad=np.random.normal(perfect_bad_data,errors)
my_bad_data_1D=Table([times,heights_bad,errors],
              names=('time','height','error'))

#print( my_data_1D )

#Write the generated data to a file on the disk of the computer
astropy.io.ascii.write( my_bad_data_1D, "1D_Generated_Data_ChiProblem.astropydat" , overwrite = True )

**Modify the data with time AND height error**
==

In [16]:
#(SECRET)
#Generate 2D uncertainty dataset
#   By 2D, we mean that we have only error in BOTH
#      the height measurement  
#      AND time measurement error 
#   The errors are not mutually exclusive. 
#   The errors can be thought of as DEPENDENT RANDOM VARIABLES

#Define the measurment error function which is something the experimentalist would give us.
covariance_matrix = np.array(
    [
        [.58,   13.35],
        [13.35, 501.77]
    ])
covariance_matrix_list = [covariance_matrix]*datapoint_count

covariance_matrices=[covariance_matrix]*len(perfect_data)

#Use the measurment error function to take the perfect data, and mess it up
times_2d=[]
heights_2d=[]
for i in range(len(perfect_data)):
    t,f=np.random.multivariate_normal(
                    mean=[times[i],perfect_data[i]],
                    cov=covariance_matrix)
    times_2d.append(t)
    heights_2d.append(f)

my_data_2D=Table([times_2d,heights_2d],names=('time','height'))

#Write the generated data to a file on the disk of the computer
astropy.io.ascii.write( my_data_2D, "2D_Generated_Data.astropydat", overwrite = True )



